# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the citibike dataset - check that you are able to read in about 40k rows

In [11]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='

try:
    from urllib.parse import urlparse, urlencode
    from urllib.request import urlopen, Request
    from urllib.error import HTTPError
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode
    from urllib2 import urlopen, Request, HTTPError

try:
    from StringIO import StringIO,BytesIO as io
except ImportError:
    from io import StringIO,BytesIO as io
    
import ast
from IPython.display import display
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urlencode({'format': format, 'q': query}).encode('utf-8')
    try:
        response = urlopen(source, data)
    except HTTPError as e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

def fetchSQL(q):
    return pd.read_csv(io(queryCartoDB(q)))

In [12]:
test = '''
SELECT *
FROM citibike_short
'''

In [13]:
df = fetchSQL(test)
df.head()

,the_geom,start_station_longitude,the_geom_webmercator,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,cartodb_id,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,-73.994811,NaN,801,2/1/2015 0:00,2/1/2015 0:14,521,8 Ave & W 31 St,40.750450,1,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978.0,2
1,NaN,-73.985649,NaN,245,2/1/2015 1:03,2/1/2015 1:07,438,St Marks Pl & 1 Ave,40.727791,89,297,E 15 St & 3 Ave,40.734232,-73.986923,20142,Subscriber,1990.0,1
2,NaN,-73.990093,NaN,379,2/1/2015 0:00,2/1/2015 0:07,497,E 17 St & Broadway,40.737050,2,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993.0,1
3,NaN,-73.973715,NaN,2474,2/1/2015 0:01,2/1/2015 0:42,281,Grand Army Plaza & Central Park S,40.764397,3,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969.0,2
4,NaN,-74.004704,NaN,818,2/1/2015 0:01,2/1/2015 0:15,2004,6 Ave & Broome St,40.724399,4,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985.0,2


# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel
#### The function is now Python 2 and 3 compatible

### TASK 1 - Familiarize with SQL Clauses

In [14]:
task = '''Sort data by start_station_id, tripduration 
    Only checking trips with duration <= 3 hours '''
print(task)
q = '''
SELECT * FROM fb55.citibike
WHERE tripduration > (60*3)
ORDER BY start_station_id, tripduration
'''
display(fetchSQL(q).head())

task = 'Only show the top/last 10 records (aka head and tail in SQL)' 
print(task)
q = '''
SELECT * FROM fb55.citibike LIMIT 10
'''
display(fetchSQL(q))

task = 'List all unique start_station_id values '
print(task)
q = '''
SELECT DISTINCT start_station_id FROM fb55.citibike
'''
display(fetchSQL(q).head())

task = '''Aggregation functions: 
    Count the number of trips (aka wc -l in SQL)
    Find the average/min/max trip duration'''
print(task)
q = '''
SELECT 
    COUNT(*) AS Total_Trips, 
    AVG(tripduration) AS Average_Trip_Duration, 
    MIN(tripduration) AS Minimum_Trip_Duration,
    MAX(tripduration) AS Maximum_Trip_Duration
FROM fb55.citibike
'''
display(fetchSQL(q).head())

Sort data by start_station_id, tripduration 
    Only checking trips with duration <= 3 hours 


,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959.0,2
1,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995.0,1
2,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970.0,1
3,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983.0,1
4,NaN,W 52 St & 11 Ave,NaN,32603,-73.993929,261,2015-02-06 08:58:00+00,2015-02-06 09:02:00+00,72,32566,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,18492,Subscriber,1971.0,1


Only show the top/last 10 records (aka head and tail in SQL)


,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0
5,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
6,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
7,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
8,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
9,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1


List all unique start_station_id values 


,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


Aggregation functions: 
    Count the number of trips (aka wc -l in SQL)
    Find the average/min/max trip duration


,total_trips,average_trip_duration,minimum_trip_duration,maximum_trip_duration
0,46200,675.865823,60,43016


### TASK 2 - Working with date/time

In [28]:
print('Selecting trips started on Feb-02-2015 only')
q = '''
SELECT * FROM fb55.citibike
WHERE starttime > '2015-02-02 00:00:00.000'
'''
display(fetchSQL(q).head())
print('''Selecting trips started on the weekends
    What are average trip duration during weekends?''')
q = '''
SELECT AVG(tripduration) AS average_trip_duration_weekends
FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime) IN (0,7)
'''
display(fetchSQL(q).head())
print('Can we do the same for weekday?')
q = '''
SELECT AVG(tripduration) AS average_trip_duration_weekdays
FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime) NOT IN (0,7)
'''
display(fetchSQL(q).head())

Selecting trips started on Feb-02-2015 only


,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
3,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
4,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1


Selecting trips started on the weekends
    What are average trip duration during weekends?


,average_trip_duration_weekends,Unnamed: 1
0,638.1492,NaN


Can we do the same for weekday?


,average_trip_duration_weekdays,Unnamed: 1
0,681.975955,NaN


### Task 3 - Working with Space

In [53]:
print('''Showing the list of start station locations
    Using GROUP BY''')
print('''Showing the number of trips started per station''')
print('''… but only for stations within 500m of Time Square!
    The coordinates of Time Square is (40.7577,-73.9857)''')

q = '''
SELECT start_station_id, MIN(start_station_name) AS name, COUNT(*) AS number_of_trips
FROM fb55.citibike
WHERE ST_Distance(
  ST_Point(start_station_longitude,start_station_latitude)::geography,
  ST_Point(-73.9857, 40.7577)::geography
) <= 500
GROUP BY start_station_id
'''
fetchSQL(q)

Showing the list of start station locations
    Using GROUP BY
Showing the number of trips started per station
… but only for stations within 500m of Time Square!
    The coordinates of Time Square is (40.7577,-73.9857)


,start_station_id,name,number_of_trips
0,173,Broadway & W 49 St,213
1,493,W 45 St & 6 Ave,141
2,2021,W 45 St & 8 Ave,141
3,529,W 42 St & 8 Ave,221
4,524,W 43 St & 6 Ave,112
5,477,W 41 St & 8 Ave,507
6,465,Broadway & W 41 St,251


### Task 4 - Putting it all together

In [56]:
print('''Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!''')
q = '''
SELECT start_station_id, MIN(start_station_name) AS name, AVG(tripduration) AS average_trip_duration
FROM fb55.citibike
WHERE ST_Distance(
  ST_Point(start_station_longitude,start_station_latitude)::geography,
  ST_Point(-73.9857, 40.7577)::geography
) <= 500
AND EXTRACT(DOW FROM starttime) IN (0,7)
GROUP BY start_station_id
ORDER BY AVG(tripduration) DESC
LIMIT 1
'''
display(fetchSQL(q))
print('''Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. ''')


Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!


,start_station_id,name,average_trip_duration
0,2021,W 45 St & 8 Ave,799.722222


Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 


In [59]:
q = '''
SELECT 
    start_station_id, MIN(start_station_name) AS start_name, 
    end_station_id, MIN(end_station_name) AS end_name, 
    COUNT(*) AS number_of_trips
FROM fb55.citibike
WHERE ST_Distance(
  ST_Point(start_station_longitude,start_station_latitude)::geography,
  ST_Point(-73.9857, 40.7577)::geography
) <= 500
AND tripduration/60 < 120
GROUP BY start_station_id, end_station_id
'''
fetchSQL(q)

,start_station_id,start_name,end_station_id,end_name,number_of_trips
0,173,Broadway & W 49 St,72,W 52 St & 11 Ave,5
1,173,Broadway & W 49 St,137,E 56 St & Madison Ave,2
2,173,Broadway & W 49 St,160,E 37 St & Lexington Ave,1
3,173,Broadway & W 49 St,164,E 47 St & 2 Ave,4
4,173,Broadway & W 49 St,168,W 18 St & 6 Ave,2
5,173,Broadway & W 49 St,173,Broadway & W 49 St,6
6,173,Broadway & W 49 St,228,E 48 St & 3 Ave,7
7,173,Broadway & W 49 St,285,Broadway & E 14 St,1
8,173,Broadway & W 49 St,300,Shevchenko Pl & E 7 St,1
9,173,Broadway & W 49 St,305,E 58 St & 3 Ave,1
